In [13]:
import pandas as pd
import numpy as np
import glob
import os

# Step 1: Find all CSV files in the current directory
csv_files = glob.glob('all_cleaned_datasets/*.csv')  # Adjust path if files are in a different directory
# Alternatively, specify files explicitly, e.g.:
# csv_files = ['filled_black_carbon.csv', 'cleaned_nitrogen_oxides.csv', 'cleaned_ari_counts.csv', ...]

In [14]:
# Step 2: Initialize an empty DataFrame for merging
combined_df = None

# Step 3: Load and merge each CSV
for file in csv_files:
    # Read the CSV
    df = pd.read_csv(file)
    
    # Ensure the CSV has exactly two columns: date and a numeric value
    if len(df.columns) != 2:
        print(f"Warning: {file} does not have exactly two columns. Skipping.")
        continue
    
    # Get the numeric column name (second column)
    value_column = df.columns[1]
    
    # Rename the value column to include the file name (without .csv) for uniqueness
    base_name = os.path.splitext(os.path.basename(file))[0]
    df = df.rename(columns={value_column: base_name})
    
    # Ensure date is in DD-MM-YYYY format
    df['date'] = pd.to_datetime(df['date'], format='mixed', dayfirst=True)    
    # Merge with combined_df
    if combined_df is None:
        combined_df = df
    else:
        combined_df = combined_df.merge(df, on='date', how='outer')



In [15]:
# Show rows that contain at least one NaN
combined_df[combined_df.isna().any(axis=1)]

,date,acute_bronchiolitis_syndromic_cd,acute_respiratory_illness_cd,black_carbon_cd,blue_particulate_matter_cd,green_particulate_matter_cd,influenza_like_syndromic_cd,infra_red_particulate_matter_cd,nitric_dioxide_cd,nitric_oxide_cd,nitrogen_oxides_as_nitrogen_dioxide_cd,Ozone_cd,PM10_cd,PM2.5_cd,red_particulate_matter_cd,scarlet_fever_syndromic_cd,UV_particulate_matter_370nm_cd,yellow_particulate_matter_590nm_cd
342,2025-05-17,7.0,150.0,NaN,NaN,NaN,3.0,NaN,13.188,3.188,18.062,82.455,15.5,9.733,NaN,1.0,NaN,NaN


In [16]:
# Step 4: Fill missing values with mean of previous 7 days
def fill_with_past_7_days_mean(row, df, column, window_days=7):
    if pd.isna(row[column]):
        start_date = row['date'] - pd.Timedelta(days=window_days)
        end_date = row['date'] - pd.Timedelta(days=1)
        past_data = df[(df['date'] >= start_date) & (df['date'] <= end_date)]
        mean_value = past_data[column].mean()
        return mean_value
    return row[column]

# Apply to each numeric column (all columns except 'date')
numeric_columns = [col for col in combined_df.columns if col != 'date']
for column in numeric_columns:
    combined_df[column] = combined_df.apply(lambda row: fill_with_past_7_days_mean(row, combined_df, column), axis=1)

In [17]:
# Round all numeric columns except 'date' to 3 decimal places
combined_df.loc[:, combined_df.columns != 'date'] = combined_df.loc[:, combined_df.columns != 'date'].round(3)

In [ ]:
# Checking if the missing value is filled correctly
combined_df[combined_df['date'] == '2025-05-17']

,date,acute_bronchiolitis_syndromic_cd,acute_respiratory_illness_cd,black_carbon_cd,blue_particulate_matter_cd,green_particulate_matter_cd,influenza_like_syndromic_cd,infra_red_particulate_matter_cd,nitric_dioxide_cd,nitric_oxide_cd,nitrogen_oxides_as_nitrogen_dioxide_cd,Ozone_cd,PM10_cd,PM2.5_cd,red_particulate_matter_cd,scarlet_fever_syndromic_cd,UV_particulate_matter_370nm_cd,yellow_particulate_matter_590nm_cd
342,2025-05-17,7.0,150.0,0.738,0.873,0.805,3.0,0.767,13.188,3.188,18.062,82.455,15.5,9.733,0.75,1.0,0.878,0.784


In [20]:
# Show rows that contain at least one NaN
combined_df[combined_df.isna().any(axis=1)]  #No more missing values are present

,date,acute_bronchiolitis_syndromic_cd,acute_respiratory_illness_cd,black_carbon_cd,blue_particulate_matter_cd,green_particulate_matter_cd,influenza_like_syndromic_cd,infra_red_particulate_matter_cd,nitric_dioxide_cd,nitric_oxide_cd,nitrogen_oxides_as_nitrogen_dioxide_cd,Ozone_cd,PM10_cd,PM2.5_cd,red_particulate_matter_cd,scarlet_fever_syndromic_cd,UV_particulate_matter_370nm_cd,yellow_particulate_matter_590nm_cd


In [21]:
# Step 5: Convert date back to DD-MM-YYYY for output
combined_df['date'] = combined_df['date'].dt.strftime('%d-%m-%Y')

In [ ]:
# Display the first few rows of the combined DataFrame
combined_df.head()

,date,acute_bronchiolitis_syndromic_cd,acute_respiratory_illness_cd,black_carbon_cd,blue_particulate_matter_cd,green_particulate_matter_cd,influenza_like_syndromic_cd,infra_red_particulate_matter_cd,nitric_dioxide_cd,nitric_oxide_cd,nitrogen_oxides_as_nitrogen_dioxide_cd,Ozone_cd,PM10_cd,PM2.5_cd,red_particulate_matter_cd,scarlet_fever_syndromic_cd,UV_particulate_matter_370nm_cd,yellow_particulate_matter_590nm_cd
0,09-06-2024,2.0,176.0,0.403,0.470,0.447,2.0,0.423,8.000,3.786,13.571,59.444,8.2,4.125,0.410,3.0,0.467,0.430
1,10-06-2024,7.0,166.0,0.337,0.373,0.353,2.0,0.347,8.500,4.214,14.929,60.900,6.1,3.000,0.327,0.0,0.363,0.347
2,11-06-2024,6.0,155.0,0.480,0.530,0.507,0.0,0.487,12.143,5.214,19.929,60.500,8.7,4.250,0.480,0.0,0.517,0.503
3,12-06-2024,2.0,157.0,0.663,0.753,0.717,0.0,0.680,14.500,6.000,23.786,64.500,10.3,4.875,0.670,0.0,0.747,0.703
4,13-06-2024,3.0,137.0,0.667,0.733,0.703,0.0,0.670,13.929,7.643,25.500,59.300,10.4,4.625,0.670,0.0,0.710,0.693


In [23]:
# Step 6: Save the combined DataFrame to a new CSV file
combined_df.to_csv('combined_data.csv', index=False)

In [24]:
# Removing '_cd' from the columns
# Step 1: Load the input CSV
input_file = 'combined_data.csv'  # Replace with your input CSV file path
df = pd.read_csv(input_file)

In [25]:
# Step 2: Rename columns by removing '_cd' (except for 'date')
new_columns = {col: col.replace('_cd', '') for col in df.columns if col != 'date'}

In [26]:
# Step 3: Apply the new column names
df = df.rename(columns=new_columns)

In [27]:
# Step 4: Save to a new CSV
output_file = 'finally_merged_datasets.csv'  # Replace with desired output file path
df.to_csv(output_file, index=False)